In [1]:
%matplotlib inline

import pandas as pd
import scanpy as sc
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse as sparse
from scipy.stats import pearsonr

from scipy.cluster.hierarchy import linkage
import scipy.spatial.distance as ssd

import seaborn as sns
import sys
from scipy.sparse import hstack, csr_matrix
from sklearn.feature_selection import mutual_info_classif
import os
from cnmf import cNMF, save_df_to_npz, load_df_from_npz
import harmonypy


from matplotlib.patches import Patch
from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize
from matplotlib.patches import Rectangle
from matplotlib.pyplot import rc_context

def save_df_to_text(obj, filename):
    obj.to_csv(filename, sep='\t')


In [2]:
sys.path.append('../../Code/')
from utils import read_dataset_log

In [3]:
from matplotlib import rcParams, font_manager
import matplotlib


rcParams['pdf.fonttype'] = 42
rcParams['ps.fonttype'] = 42
rcParams['font.family'] = 'sans-serif'
rcParams['font.sans-serif'] = ['Arial']

In [4]:
tcat_dir = '../../Data/TCAT/'

In [5]:
params = read_dataset_log('Current Dataset Paths')
params.index = params['dataset']
params

,dataset,usage_fn,gene_scores_fn,gene_tpm_fn,tcat_fn,manual_gating_fn,metadata_fn,Processing notebook path,cNMF notebook path,cluster_col,...,processed_forcnmf_fn,raw_counts_fn,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23
dataset,,,,,,,,,,,,,,,,,,,,,
AMP-RA,AMP-RA,/data/srlab1/mcurtis/GSK/tcell_proliferation/A...,/data/srlab1/mcurtis/GSK/tcell_proliferation/A...,/data/srlab1/mcurtis/GSK/tcell_proliferation/A...,/data/srlab1/TCAT/Data/TCAT/TCAT_Usage_2023070...,NaN,/data/srlab1/mcurtis/GSK/tcell_proliferation/A...,/data/srlab1/mcurtis/GSK/tcell_proliferation/n...,/data/srlab1/mcurtis/GSK/tcell_proliferation/n...,cluster_name,...,/data/srlab1/mcurtis/GSK/tcell_proliferation/A...,/data/srlab1/mcurtis/GSK/tcell_proliferation/A...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Pan-Cancer,Pan-Cancer,/data/srlab1/mcurtis/GSK/tcell_proliferation/z...,/data/srlab1/mcurtis/GSK/tcell_proliferation/z...,/data/srlab1/mcurtis/GSK/tcell_proliferation/z...,/data/srlab1/TCAT/Data/TCAT/TCAT_Usage_2023070...,NaN,/data/srlab1/mcurtis/GSK/tcell_proliferation/z...,/data/srlab1/mcurtis/GSK/tcell_proliferation/n...,/data/srlab1/mcurtis/GSK/tcell_proliferation/n...,meta.cluster,...,/data/srlab1/mcurtis/GSK/tcell_proliferation/z...,/data/srlab1/mcurtis/GSK/tcell_proliferation/z...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TBRU,TBRU,/data/srlab1/mcurtis/GSK/tcell_proliferation/T...,/data/srlab1/mcurtis/GSK/tcell_proliferation/T...,/data/srlab1/mcurtis/GSK/tcell_proliferation/T...,/data/srlab1/TCAT/Data/TCAT/TCAT_Usage_2023070...,NaN,/data/srlab1/mcurtis/GSK/tcell_proliferation/T...,/data/srlab1/mcurtis/GSK/tcell_proliferation/n...,/data/srlab1/mcurtis/GSK/tcell_proliferation/n...,cluster_name,...,/data/srlab1/mcurtis/GSK/tcell_proliferation/T...,/data/srlab1/mcurtis/GSK/tcell_proliferation/T...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
HIV-Vaccine,HIV-Vaccine,/data/srlab1/dk718/Tcell_cNMF_Comparison/Data/...,/data/srlab1/dk718/Tcell_cNMF_Comparison/Data/...,/data/srlab1/dk718/Tcell_cNMF_Comparison/Data/...,/data/srlab1/TCAT/Data/TCAT/TCAT_Usage_2023070...,/data/srlab1/TCAT/Data/PerDataset/HaoEtAl/Manu...,/data/srlab1/dk718/Tcell_cNMF_Comparison/Data/...,/data/srlab1/dk718/Tcell_cNMF_Comparison/Analy...,/data/srlab1/dk718/Tcell_cNMF_Comparison/Analy...,celltype.l2,...,/data/srlab1/TCAT/Data/PerDataset/HaoEtAl/haoe...,/data/srlab1/TCAT/Data/PerDataset/HaoEtAl/haoe...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UK-Covid,UK-Covid,/data/srlab1/mcurtis/GSK/tcell_proliferation/C...,/data/srlab1/mcurtis/GSK/tcell_proliferation/C...,/data/srlab1/mcurtis/GSK/tcell_proliferation/C...,/data/srlab1/TCAT/Data/TCAT/TCAT_Usage_2023070...,NaN,/data/srlab1/dk718/COVID_PBMC/Data/Haniffa/han...,/data/srlab1/dk718/Tcell_cNMF_Comparison/Analy...,/data/srlab1/dk718/Tcell_cNMF_Comparison/Analy...,full_clustering,...,/data/srlab1/dk718/COVID_PBMC/Data/Haniffa/han...,/data/srlab1/TCAT/Data/PerDataset/UKCOVID/hani...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
COMBAT,COMBAT,/data/srlab1/mcurtis/GSK/tcell_proliferation/C...,/data/srlab1/mcurtis/GSK/tcell_proliferation/C...,/data/srlab1/mcurtis/GSK/tcell_proliferation/C...,/data/srlab1/TCAT/Data/TCAT/TCAT_Usage_2023070...,/data/srlab1/TCAT/Data/PerDataset/COMBAT/Manua...,/data/srlab1/dk718/Tcell_cNMF_Comparison/Data/...,/data/srlab1/dk718/Tcell_cNMF_Comparison/Analy...,/data/srlab1/dk718/Tcell_cNMF_Comparison/Analy...,Annotation_minor_subset,...,/data/srlab1/dk718/Tcell_cNMF_Comparison/Data/...,/data/srlab1/TCAT/Data/PerDataset/COMBAT/COMBA...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Pan-Tissue,Pan-Tissue,/data/srlab1/dk718/Tcell_cNMF_Comparison/Data/...,/data/srlab1/dk718/Tcell_cNMF_Comparison/Data/...,/data/srlab1/dk718/Tcell_cNMF_Comparison/Data/...,/data/srlab1/TCAT/Data/TCAT/TCAT_Usage_2023070...,NaN,/data/srlab1/dk718/Tcell_cNMF_Comparison/Data/...,/data/srlab1/dk718/Tcell_cNMF_Comparison/Analy...,/data/srlab1/dk718/Tcell_cNMF_Comparison/Analy...,Manually_curated_celltype,...,/data/srlab1/dk718/Tcell_cNMF_Comparison/Data/...,/data/srlab1/dk71

In [28]:
clus_dict = {}

for cluster_method in ['Z_HVGunion', 'TPMnorm.HGVunion']:
    clus_fn = tcat_dir + 'cluster_groups.pairwiseCorr.%s.20230911.txt' % cluster_method
    clus_df = pd.read_csv(clus_fn, index_col = 0, sep = '\t').loc[:, params.index]
    clus_dict[cluster_method] = clus_df

In [29]:
clus_dict['Z_HVGunion'].columns

Index(['AMP-RA', 'Pan-Cancer', 'TBRU', 'HIV-Vaccine', 'UK-Covid', 'COMBAT',
       'Pan-Tissue'],
      dtype='object', name='dataset')

In [37]:
clus_dict[cluster_method].head(50)

dataset,AMP-RA,Pan-Cancer,TBRU,HIV-Vaccine,UK-Covid,COMBAT,Pan-Tissue
CellCycle-G2M_CellCycle_G2M,AMP-RA:CellCycle-G2M,Pan-Cancer:CellCycle-G2M,TBRU:CellCycle_G2M,HIV-Vaccine:CellCycle_G2M,UK-Covid:CellCycle_G2M,COMBAT:CellCycle_G2M,Pan-Tissue:CellCycle-G2M
Mito,NaN,NaN,NaN,HIV-Vaccine:Mito,UK-Covid:Mito,COMBAT:Mito,Pan-Tissue:Mito
ISG,AMP-RA:ISG,Pan-Cancer:ISG,TBRU:ISG,HIV-Vaccine:ISG,UK-Covid:ISG,COMBAT:ISG,Pan-Tissue:ISG
Ribo_Translation,AMP-RA:Ribo,NaN,TBRU:Ribo,HIV-Vaccine:Translation,UK-Covid:Translation,COMBAT:Translation,Pan-Tissue:Translation
Doublet_DC_HLA,AMP-RA:Doublet_DC,Pan-Cancer:HLA,TBRU:HLA,HIV-Vaccine:HLA,UK-Covid:HLA,COMBAT:HLA,Pan-Tissue:HLA
CellCycle-Sphase_CellCycle_S_DTL/FAM111B,AMP-RA:CellCycle-Sphase,Pan-Cancer:CellCycle-Sphase,TBRU:DTL/FAM111B,HIV-Vaccine:CellCycle_S,UK-Covid:CellCycle_S,COMBAT:CellCycle_S,Pan-Tissue:CellCycle-Sphase
gdT,NaN,NaN,TBRU:gdT,HIV-Vaccine:gdT,UK-Covid:gdT,COMBAT:gdT,NaN
HeatShock,AMP-RA:HeatShock,Pan-Cancer:HeatShock,NaN,NaN,NaN,NaN,Pan-Tissue:HeatShock
Metallothionein,AMP-RA:Metallothionein,Pan-Cancer:Metallothionein,NaN,NaN,UK-Covid:Metallothionein,COMBAT:Metallothionein,Pan-Tissue:Metallothionein
CellCycle-Histone_CellCycle-Histone2_CellCycle_Histone,AMP-RA:CellCycle-Histone,Pan-Cancer:CellCycle-Histone2,TBRU:CellCycle_Histone,HIV-Vaccine:CellCycle_Histone,UK-Covid:CellCycle_Histone,COMBAT:CellCycle_Histone,Pan-Tissue:CellCycle-Histone


In [100]:
def top_pairs(clus, clus2):
    diff_prg = pd.DataFrame(0, index = clus.index, columns = clus2.index)

    for prg in clus.index:
        for prg2 in clus2.index:
            # print(prg2)
            subprgs = set(clus.loc[prg, ].dropna())
            subprgs2 = set(clus2.loc[prg2, ].dropna())
            diff_prg.loc[prg, prg2] = len(subprgs.difference(subprgs2) | subprgs2.difference(subprgs))
            
    diff_prg_pairs = pd.DataFrame(diff_prg.unstack().sort_values(ascending = True), 
                                ).reset_index()
    diff_prg_pairs.columns =['prg2', 'prg1', 'diff']

    # ### First occurrence of each prg similarity to another
    # top_pairs = diff_prg_pairs.loc[sorted(list(set(diff_prg_pairs.drop_duplicates('prg1').index).union(diff_prg_pairs.drop_duplicates('prg2').index))),
    #                   ]
    top_pairs = diff_prg_pairs
    
    return top_pairs

In [101]:
pairs = top_pairs(clus_dict['Z_HVGunion'], clus_dict['TPMnorm.HGVunion'])

In [103]:
pairs_unique = pairs.loc[sorted(list(
    set(pairs.drop_duplicates('prg1').index).union(pairs.drop_duplicates('prg2').index))), ]


In [105]:
for n in range(0, len(params.index)):
    print(n, 
          pairs_unique[(pairs_unique['diff']==n) & (["Single" not in x for x in pairs_unique['prg1']])].shape[0])
    

0 34
1 9
2 4
3 2
4 1
5 0
6 0


In [106]:
pairs_unique.tail(40)

,prg2,prg1,diff
84,UK-Covid:KRT17/DDAH1_Single,UK-Covid:KRT17/DDAH1_Single,0
85,COMBAT:TYMS/ATP5F1D_Single,COMBAT:TYMS/ATP5F1D_Single,0
86,COMBAT:CellCycle_G2M_2_Single,COMBAT:CellCycle_G2M_2_Single,0
87,COMBAT:CellCycle_Histone3_Single,COMBAT:CellCycle_Histone3_Single,0
88,COMBAT:Doublet_Plasmablast2_Single,COMBAT:Doublet_Plasmablast2_Single,0
89,COMBAT:CDO1/IGFL2_Single,COMBAT:CDO1/IGFL2_Single,0
90,COMBAT:SFRP4/CADM1_Single,COMBAT:SFRP4/CADM1_Single,0
91,UK-Covid:CRHBP/EGFL7_Single,UK-Covid:CRHBP/EGFL7_Single,0
92,Mito,Mito,0
93,Pan-Tissue:Doublet_Plasmablast3_Single,Pan-Tissue:Doublet_Plasmablast3_Single,0


In [62]:
sorted(clus_dict['Z_HVGunion'].index)

['?AB_CD278/ACTG1_AB_CD278/ACTG1',
 '?AB_CD278/ACTG1_CD4_CM-1_LIMS1/PASK',
 '?ANXA1/MYADM_?CCR10/LMNA_?IL32/S100A_CD4_Memory_LGALS1/S100A10',
 '?BCL2/CDK6_BCL2/CDK6',
 '?Doublet_Myeloid_?GutMyeloid2_MMP8_Doublet_Myeloid',
 '?PlasmaDoublet/FCN3/FCN2/OIT3_CRHBP/CLEC4G',
 '?RGCC/MYADM/TNFAIP3_KLF6/CXCR4',
 '?SOX4/MME_SOX4/MME',
 'AB_CD278/LGMN/IL21_LAG3/CD38',
 'AB_CD86/TIMD4_dnT1/CD38',
 'AMP-RA:Doublet_Fibroblast_Single',
 'AMP-RA:Doublet_Mast_Single',
 'AMP-RA:Doublet_Plasma_Single',
 'AMP-RA:Doublet_cardiomyocyte_Single',
 'AMP-RA:Doublet_pDC_Single',
 'AMP-RA:Hypoxia_Single',
 'AMP-RA:SERPINE2_Single',
 'AMP-RA:Tph_Single',
 'AMP-RA_CD38_dnT_dnT/IL10+CD38+',
 'CD4_CM-2_PTPN13/TNFRSF4_Th17',
 'CD4_Naive_Ribo/T_Naive_T_Naive_Translation_Translation/T_Naive',
 'CD8A/GNLY_Memory',
 'CD8_CD160high_CD8_class2_HLA',
 'CD8_Naive',
 'COMBAT:?Myeloid_Single',
 'COMBAT:?PGLYRP1/LTF_Single',
 'COMBAT:CDO1/IGFL2_Single',
 'COMBAT:CellCycle_G2M_2_Single',
 'COMBAT:CellCycle_Histone2_Single',
 'COM

In [63]:
clus_dict['Z_HVGunion'].loc['CD4_Naive_Ribo/T_Naive_T_Naive_Translation_Translation/T_Naive', :]

dataset
AMP-RA                    AMP-RA:Ribo/T_Naive
Pan-Cancer     Pan-Cancer:Translation/T_Naive
TBRU                                      NaN
HIV-Vaccine             HIV-Vaccine:CD4_Naive
UK-Covid                   UK-Covid:CD4_Naive
COMBAT                     COMBAT:Translation
Pan-Tissue                 Pan-Tissue:T_Naive
Name: CD4_Naive_Ribo/T_Naive_T_Naive_Translation_Translation/T_Naive, dtype: object

In [65]:
clus_dict['TPMnorm.HGVunion'].loc['CD4_Naive_Ribo/T_Naive_T_Naive_Translation/T_Naive', :]

dataset
AMP-RA                    AMP-RA:Ribo/T_Naive
Pan-Cancer     Pan-Cancer:Translation/T_Naive
TBRU                             TBRU:T_Naive
HIV-Vaccine             HIV-Vaccine:CD4_Naive
UK-Covid                   UK-Covid:CD4_Naive
COMBAT                                    NaN
Pan-Tissue                 Pan-Tissue:T_Naive
Name: CD4_Naive_Ribo/T_Naive_T_Naive_Translation/T_Naive, dtype: object

In [67]:
clus_dict['Z_HVGunion'].loc['Ribo_Translation', :]

dataset
AMP-RA                     AMP-RA:Ribo
Pan-Cancer                         NaN
TBRU                         TBRU:Ribo
HIV-Vaccine    HIV-Vaccine:Translation
UK-Covid          UK-Covid:Translation
COMBAT                             NaN
Pan-Tissue      Pan-Tissue:Translation
Name: Ribo_Translation, dtype: object

In [66]:
clus_dict['TPMnorm.HGVunion'].loc['Ribo_Translation', :]

dataset
AMP-RA                     AMP-RA:Ribo
Pan-Cancer                         NaN
TBRU                         TBRU:Ribo
HIV-Vaccine    HIV-Vaccine:Translation
UK-Covid          UK-Covid:Translation
COMBAT              COMBAT:Translation
Pan-Tissue      Pan-Tissue:Translation
Name: Ribo_Translation, dtype: object

In [71]:
clus_dict['Z_HVGunion'].loc['Tfh', :]

dataset
AMP-RA             AMP-RA:Tfh
Pan-Cancer     Pan-Cancer:Tfh
TBRU                      NaN
HIV-Vaccine               NaN
UK-Covid                  NaN
COMBAT                    NaN
Pan-Tissue     Pan-Tissue:Tfh
Name: Tfh, dtype: object

In [72]:
clus_dict['TPMnorm.HGVunion'].loc['SERPINE2_Tfh', :]

dataset
AMP-RA         AMP-RA:SERPINE2
Pan-Cancer      Pan-Cancer:Tfh
TBRU                       NaN
HIV-Vaccine                NaN
UK-Covid                   NaN
COMBAT                     NaN
Pan-Tissue      Pan-Tissue:Tfh
Name: SERPINE2_Tfh, dtype: object

In [78]:
clus_dict['Z_HVGunion'].loc['IEG_IEG2', :]

dataset
AMP-RA             AMP-RA:IEG
Pan-Cancer     Pan-Cancer:IEG
TBRU                      NaN
HIV-Vaccine               NaN
UK-Covid        UK-Covid:IEG2
COMBAT                    NaN
Pan-Tissue     Pan-Tissue:IEG
Name: IEG_IEG2, dtype: object

In [79]:
clus_dict['TPMnorm.HGVunion'].loc['IEG_IEG2', :]

dataset
AMP-RA             AMP-RA:IEG
Pan-Cancer     Pan-Cancer:IEG
TBRU                      NaN
HIV-Vaccine               NaN
UK-Covid        UK-Covid:IEG2
COMBAT             COMBAT:IEG
Pan-Tissue     Pan-Tissue:IEG
Name: IEG_IEG2, dtype: object

In [84]:
clus_dict['Z_HVGunion'].loc['YPEL5/CREM', :]

dataset
AMP-RA             AMP-RA:YPEL5/CREM
Pan-Cancer     Pan-Cancer:YPEL5/CREM
TBRU                             NaN
HIV-Vaccine                      NaN
UK-Covid                         NaN
COMBAT                           NaN
Pan-Tissue                       NaN
Name: YPEL5/CREM, dtype: object

In [85]:
clus_dict['TPMnorm.HGVunion'].loc['IEG_IEG1_YPEL5/CREM', :]

dataset
AMP-RA             AMP-RA:YPEL5/CREM
Pan-Cancer     Pan-Cancer:YPEL5/CREM
TBRU                        TBRU:IEG
HIV-Vaccine          HIV-Vaccine:IEG
UK-Covid               UK-Covid:IEG1
COMBAT                           NaN
Pan-Tissue                       NaN
Name: IEG_IEG1_YPEL5/CREM, dtype: object

In [98]:
clus_dict['Z_HVGunion'].loc['Pan-Tissue:?Plasmablast_Single', :]

dataset
AMP-RA                             NaN
Pan-Cancer                         NaN
TBRU                               NaN
HIV-Vaccine                        NaN
UK-Covid                           NaN
COMBAT                             NaN
Pan-Tissue     Pan-Tissue:?Plasmablast
Name: Pan-Tissue:?Plasmablast_Single, dtype: object

In [ ]:
clus_dict['TPMnorm.HGVunion'].loc['Pan-Tissue:?Plasmablast_Single', :]

~30 of the 50 non-singleton pairs are identical between Zspec and TPMnorm. Differences largely make sense (i.e. Naive GEP replaced for a translation GEP). Some different merging of IEG GEPs.